In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# Charger le dataset
df = pd.read_csv('./final_dataset.csv')

In [3]:
# Nombre de classes
n_classes = df['activityID'].nunique()
n_classes = 25
n_total = 3000000

# Taille de l'échantillon par classe
n_per_class = n_total // n_classes

# Échantillonnage uniforme
sample = df.groupby('activityID', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), n_per_class), random_state=42)
)

# Réinitialiser les indices
sample = sample.reset_index(drop=True)

/var/folders/xz/0tnfv35j1jjbznbfc7k0swh00000gn/T/ipykernel_15434/178802134.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample = df.groupby('activityID', group_keys=False).apply(


In [4]:
sample['activityID'].value_counts()

activityID
0     120000
7     120000
19    120000
17    120000
16    120000
1     120000
10    120000
6     120000
4     120000
3     120000
2     120000
12    117216
13    104944
18     99878
5      98199
9      83646
11     54519
24     49360
20     46915
Name: count, dtype: int64

In [5]:
# Préparer les données
X = sample.drop(columns=["activityID", "timestamp"])  # Supprimer les colonnes inutiles
y = sample["activityID"]

In [6]:
# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construction du modèle de réseau de neurones
# Création du modèle
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(n_classes, activation='softmax')  # Pour une classification multi-classes
])

# Compilation du modèle
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2, verbose=1)


Epoch 1/50
9874/9874 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.6249 - loss: 1.2874 - val_accuracy: 0.8760 - val_loss: 0.3792
Epoch 2/50
9874/9874 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.8299 - loss: 0.5295 - val_accuracy: 0.8983 - val_loss: 0.3012
Epoch 3/50
9874/9874 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.8507 - loss: 0.4659 - val_accuracy: 0.9124 - val_loss: 0.2661
Epoch 4/50
9874/9874 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.8607 - loss: 0.4341 - val_accuracy: 0.9165 - val_loss: 0.2525
Epoch 5/50
9874/9874 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.8676 - loss: 0.4131 - val_accuracy: 0.9194 - val_loss: 0.2408
Epoch 6/50
9874/9874 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.8728 - loss: 0.3985 - val_accuracy: 0.9227 - val_loss: 0.2288
Epoch 7/50
9874/9874 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.8764 - loss: 0.3870 - val_accuracy: 0.9276 - val_loss: 0.2161
Epoch 8/50
9874/9874 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.8800 - loss: 0

In [8]:
y_pred = model.predict(X_test)

12342/12342 ━━━━━━━━━━━━━━━━━━━━ 3s 232us/step


In [9]:

y_pred_classes = np.argmax(y_pred, axis=1)
print(y_pred_classes)

[ 3  4  4 ... 18  6  1]


In [10]:
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_classes))

Accuracy: 95.44%

Classification Report:

              precision    recall  f1-score   support

           0       0.84      0.61      0.71     23874
           1       1.00      0.97      0.99     23903
           2       0.99      0.98      0.99     23930
           3       0.95      0.99      0.97     24037
           4       0.93      0.96      0.94     24105
           5       0.99      0.98      0.98     19848
           6       0.98      0.99      0.99     24002
           7       0.96      0.96      0.96     23981
           9       1.00      0.99      1.00     16764
          10       1.00      0.99      1.00     23935
          11       1.00      1.00      1.00     10945
          12       0.89      0.97      0.93     23442
          13       0.90      0.96      0.92     21288
          16       0.95      0.97      0.96     24069
          17       0.96      0.97      0.97     23745
          18       0.96      0.98      0.97     19785
          19       0.96      0.97      

In [11]:
import numpy
print(numpy.unique(y_train))

[ 0  1  2  3  4  5  6  7  9 10 11 12 13 16 17 18 19 20 24]


In [12]:
import numpy as np

x = np.array([[101.0,34.5625,-5.90031,4.90636,3.1681,-5.81862,5.227,3.44635,0.284144,0.0166734,-0.313876,47.6911,-41.2161,3.94013,1.0,0.0,0.0,0.0,37.3125,-0.104606,-4.0706,-9.433,-0.379479,-4.11858,-8.96098,0.0583489,0.0208029,-0.160818,-10.319,41.947,43.0069,1.0,0.0,0.0,0.0,36.375,9.01379,-3.60497,-2.67982,9.0058,-3.52498,-1.99554,0.0426612,0.0112998,0.055789,-63.2542,-0.971412,3.54882,1.0,0.0,0.0,0.0]])
#print(x)
y_pred = model.predict(x)
y_pred_classes = np.argmax(y_pred, axis=1)
print(y_pred_classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[18]


In [13]:
print(y_pred)

[[7.9109217e-05 1.9588556e-15 9.8351038e-23 7.8485727e-20 4.3274556e-27
  6.6132307e-32 4.8801242e-32 8.4781839e-33 0.0000000e+00 1.3723647e-18
  1.1638954e-15 8.4217193e-16 9.3565036e-26 2.3131121e-22 0.0000000e+00
  0.0000000e+00 2.8135275e-17 9.0254820e-10 9.9992085e-01 5.3022415e-08
  5.0999043e-34 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]]


In [18]:
# Save the model in HDF5 format
model.save("model.h5")

In [16]:
print(y_pred)

[[7.9109217e-05 1.9588556e-15 9.8351038e-23 7.8485727e-20 4.3274556e-27
  6.6132307e-32 4.8801242e-32 8.4781839e-33 0.0000000e+00 1.3723647e-18
  1.1638954e-15 8.4217193e-16 9.3565036e-26 2.3131121e-22 0.0000000e+00
  0.0000000e+00 2.8135275e-17 9.0254820e-10 9.9992085e-01 5.3022415e-08
  5.0999043e-34 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]]
